<a href="https://colab.research.google.com/github/MarcChen/Fewshot-ICL/blob/main/CHEN_Marc_SEBTI_Adam_TableGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: TableGPT


In this lab, we'll discover the power of code generation models through TableGPT2. The aim is to see how the model can be used in data analysis.

First of all, the notebook is divided into X sections:
0. Installation: This section is dedicated to module installation, model loading and data loading.
1. Guided introduction: Together, we'll discover how to use and evaluate TableGPT2.
2. More questions: You'll need to add at least one new question type to our simple evaluation system.
3. More data sets: You'll need to implement a question with multiple datasets.


IMPORTANT:
- You must work in pairs. You must submit **ONLY ONE NOTEBOOK** for each pair.
- Do not share your work with other pairs.
- You should not use Copilot, ChatGPT or similar tools. At the very least, remove the prompt ...
- <font color='red'>All the things you need to do are indicated in red.</font>


<font color='red'>**FIRST QUESTION:** What are the specificty of the TableGPT2 model?</font> https://huggingface.co/tablegpt/TableGPT2-7B

###Q1 : What are the specificity of TableGPT2

TableGPT2-7B is a specialized model designed to handle tabular data and generate code or insights for data analysis tasks. It processes tables formatted like a DataFrame and provides text-based outputs, making it ideal for tasks such as BI question answering and generating analysis scripts. Built on the Qwen2.5 architecture, it has been fine-tuned with millions of examples to excel in real-world applications.

## 0. Setup

In [1]:
!pip install transformers datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

import pandas as pd
import torch

In [3]:
llm_name = "tablegpt/TableGPT2-7B"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left")
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
)
# Set LLM on eval mode.
llm.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-0

In [4]:
generation_config = GenerationConfig(
  max_new_tokens=512,
  do_sample=False,
  # do_sample=True,
  # temperature=.7,
  # top_p=.8,
  # top_k=20,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
)

In [5]:
df = pd.read_csv("hf://datasets/phihung/titanic/train.csv")
df = df.drop("Cabin", axis=1).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 66.8+ KB


## 1.1 Guided Introduction: The Model.

Below there is an example of a prompt that could be used with TableGPT2.

```
Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Henry   male 35.0      0      0           373450  8.0500        S
*/

Question: How many child survive? (under 18)
```

The prompt is divided in 3 parts:
1. The global instruction wich is to write python that could answer a question on a specific dataset.
2. The header of the given dataset: 5 first lines of titanic dataset.
3. The question to answer: "How many child survive? (under 18)


First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `generate_answer` function following the comments inside.</font>


In [51]:
example_prompt_template = """Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""

def generate_answer(prompt, llm=llm, generation_config=generation_config):
  # Code taken in the quickstart of TableGPT2 https://huggingface.co/tablegpt/TableGPT2-7B

  messages = [
    {"role": "system", "content": "You are a helpful assistant to write some python code to solve problems on pandas dataframe."},
    {"role": "user", "content": prompt},
  ]

  text = tokenizer.apply_chat_template(
      messages, tokenize=False, add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(llm.device)

  generated_ids = llm.generate(**model_inputs, max_new_tokens=512, generation_config=generation_config)
  generated_ids = [
      output_ids[len(input_ids) :]
      for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return response




prompt = example_prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question="How many child survive? (under 18)",
)

answer = generate_answer(prompt)

print(prompt)
print("\n*****\n")
print(answer)

Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Hen

### Test of the provided output thanks to TableGPT2 answer given abbove this cell

In [10]:
import pandas as pd

# Assuming the dataframe is named 'df'
# Filter the dataframe to include only passengers under 18 who survived
child_survivors = df[(df['Age'] < 18) & (df['Survived'] == 1)]

# Count the number of child survivors
num_child_survivors = child_survivors.shape[0]

# Output the result
output = num_child_survivors
print(output)

61


## 1.2 Guided Introduction: The Answer.

As you can see, the model answer with some generated code.

```
Python code:
```python
# Filter the dataframe to include only passengers under the age of 18
children = df[df['Age'] < 18]

# Count the number of children who survived
child_survivors = children[children['Survived'] == 1]

# Save the answer in the variable output
output = len(child_survivors)
```

So we will need to execute it, but there is some difficulty:
1. Sometime, the llm answer with \`\`\`python ... \`\`\`, sometime the llm answer directly with the code. We need to handle both cases.
2. We need to recover the variable output from the execution.
3. We need to evaluate single value and list of values.


First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `exec_answer` function following the comments inside.</font>


In [63]:
import re

def exec_answer(answer, gold, debug=False):
    try:
        # Extract the code from the answer. Be careful, the code is now always in ``` ```.
        reg = re.search(r"```python(.*?)```", answer, re.DOTALL)
        if not reg:
            code = answer
        else :
          code = reg.group(1)

        local_vars = {'df': df}
        # Execute the code, https://docs.python.org/3/library/functions.html#exec
        exec(code, {}, local_vars)

        # if the code work: Return True or False based on output == gold (be careful to handle iterable !)

        print(f"Generated output : {local_vars.get('output')}") if debug else None
        print(f"Gold output : {gold}") if debug else None
        return local_vars.get("output") == gold
    except Exception as e:
        # if the code don't work return False.
        print(f"\033[93mExec answer function failed - exception {e} \033[0m")
        return False


print(exec_answer(answer, 61))

61
True


## 1.3 Guided Introduction: The Question.

Now we want to automatically generate questions to evaluate the performance of our model. There are benchmarks on this subject, but here we want to practice code by generating the questions ourselves.

We will generate some basic filter questions.

<font color='red'>TODO: Fill in the `generate_filter_question` function following the comments inside.</font>


### Question we ask : What's the maximum value of {target_column} where {filter_column} == {filter_value}?

In [91]:
import random

def generate_random_question(generate_function, df, k=1, seed=42):
  random.seed(seed)
  return [generate_function(df) for _ in range(k)]

def generate_filter_question(df):
    # Create a question template that takes a target column, a filter column, and a filter value

    ### We're computing here the maximum value of the target column with the filter column and filter value ###

    numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
    columns = df.columns.tolist()

    target_column = random.choice(numeric_columns)
    filter_column = random.choice(columns)

    # Target & Filter columns should be different
    while filter_column == target_column:
        filter_column = random.choice(columns)

    # Get a random filter value inside the filter column. Avoid NaN values.
    non_nan_values = df[filter_column].dropna().unique()

    filter_value = random.choice(non_nan_values)

    # Compute the correct answer for the given target column, filter column, and filter value.
    filtered_df = df[df[filter_column] == filter_value]

    if filtered_df.empty:
        answer = None
    else:
        answer = filtered_df[target_column].max()  # Example calculation: max of target column

    # Return formatted question and associated answer in a dict {"question": [question], "answer": [answer]}
    question = (f"What's the maximum value of {target_column} where {filter_column} == {filter_value}?")

    return {"question": question, "answer": answer}

generate_random_question(generate_filter_question, df, k=5)

[{'question': "What's the maximum value of Parch where Survived == 0?",
  'answer': 6},
 {'question': "What's the maximum value of Parch where Sex == male?",
  'answer': 5},
 {'question': "What's the maximum value of Survived where Pclass == 2?",
  'answer': 1},
 {'question': "What's the maximum value of PassengerId where Embarked == Q?",
  'answer': 891},
 {'question': "What's the maximum value of SibSp where Survived == 1?",
  'answer': 4}]

## 1.4 Guided Introduction: The Evaluation.

The last step in this section is to evaluate our model on 20 random questions! We'll use simple accuracy.

You should have an accuracy between 0.9 and 1.

<font color='red'>TODO: Follow instruction in comment of the cell below.</font>

<font color='green'>BONUS: Investigate on errors and improve our prompt/parsing to solve them.</font>


### It kept creating new dataframe for example as is the dataframe given in the prompt didn't exist. So we emphasized a bit more on the existing dataframe named {var_name}

In [68]:
prompt_template = """Given access to pandas dataframe named {var_name} which already exist, write the Python code to answer the user's question.
The answer should be store in a variable named "output". Here is a sample of the existing dataframe named {var_name}:

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""

In [87]:
from tqdm import tqdm

# Generate 20 random questions
num_question = 20
correct = 0

questions = generate_random_question(generate_filter_question, df, k=num_question)

# Iterate over question to format prompt, generate answer and execute answer.
for q in tqdm(questions):
    question = q["question"]
    expected_answer = q["answer"]
    # print(f"Expected answer {expected_answer}")
    # print(f"Question: {question}")

    prompt = prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question=question,
)

    generated_answer = generate_answer(prompt)
    # print(f"Generated_answer is : {generated_answer}")

    bool_result = exec_answer(str(generated_answer), int(expected_answer), debug=False)

    if bool_result:
        correct += 1
        print(f"\033[93m Correctly answered \033[0m")

# Report the accuracy
accuracy = correct / len(questions)
print(f"\033[91mAcc: {accuracy * 100 :.2f}\033[0m")


  5%|▌         | 1/20 [00:06<02:10,  6.85s/it]

 Correctly answered 


 10%|█         | 2/20 [00:12<01:55,  6.39s/it]

 Correctly answered 


 15%|█▌        | 3/20 [00:20<01:54,  6.74s/it]

...

 Correctly answered 


 95%|█████████▌| 19/20 [03:37<00:09,  9.04s/it]

 Correctly answered 


100%|██████████| 20/20 [03:45<00:00, 11.28s/it]

 Correctly answered 
Acc: 80.00


## 2. More Questions.

Now it's your turn to imagine a type of question ("How many ..."). Implement a function to generate new type of question. Verify that our previous code work with your new question then evaluate it.

<font color='red'>TODO: Generate **AT LEAST ONE** new type of question and report this new question accuracy.</font>


# Question generated is now :     question = "How many distinct values of {target_column} where {filter_column} == {filter_value}?"


In [92]:
def generate_filter_count_distinct_question(df):
    # Get a random target column and filter column
    columns = df.columns.tolist()
    target_column = random.choice(columns)
    filter_column = random.choice(columns)

    while filter_column == target_column:
        filter_column = random.choice(columns)

    # Get a random filter value inside the filter column. Avoid NaN values.
    non_nan_values = df[filter_column].dropna().unique()

    filter_value = random.choice(non_nan_values)

    # Compute the correct answer: count distinct values in the target column that meet the condition
    filtered_df = df[df[filter_column] == filter_value]
    answer = filtered_df[target_column].nunique()

    # Return formatted question and associated answer in a dict {"question": [question], "answer": [answer]}
    question = (f"How many distinct values of {target_column} where {filter_column} == {filter_value}?")

    return {"question": question, "answer": answer}


generate_random_question(generate_filter_count_distinct_question, df, k=5)

[{'question': 'How many distinct values of Embarked where Survived == 0?',
  'answer': 3},
 {'question': 'How many distinct values of Sex where Name == Stankovic, Mr. Ivan?',
  'answer': 1},
 {'question': 'How many distinct values of Pclass where Survived == 0?',
  'answer': 3},
 {'question': 'How many distinct values of Fare where SibSp == 1?',
  'answer': 94},
 {'question': 'How many distinct values of PassengerId where Survived == 0?',
  'answer': 424}]

In [86]:
questions = generate_random_question(generate_filter_count_distinct_question, df, k=num_question)

# Iterate over questions to format prompt, generate answer, and compute accuracy
correct = 0
for q in tqdm(questions):
    question = q["question"]
    expected_answer = q["answer"]
    # print(f"Expected answer {expected_answer}")
    # print(f"Question: {question}")

    prompt = prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question=question,
)

    generated_answer = generate_answer(prompt)
    # print(f"Generated_answer is : {generated_answer}")

    bool_result = exec_answer(str(generated_answer), int(expected_answer), debug=False)

    if bool_result:
        correct += 1
        print(f"\033[93m Correctly answered \033[0m")

# Report the accuracy
accuracy = correct / len(questions)
print(f"\033[91mAcc: {accuracy * 100 :.2f}\033[0m")

  5%|▌         | 1/20 [00:09<03:06,  9.81s/it]

 Correctly answered 


 10%|█         | 2/20 [00:18<02:45,  9.22s/it]

 Correctly answered 


 15%|█▌        | 3/20 [00:26<02:23,  8.45s/it]

...

 Correctly answered 


 95%|█████████▌| 19/20 [02:32<00:07,  7.72s/it]

 Correctly answered 


100%|██████████| 20/20 [02:40<00:00,  8.00s/it]

 Correctly answered 
Acc: 100.00


## 3. More datasets.

Below we load a new dataset: "adult_income_dataset".

<font color='red'>TODO: Evaluate our questions on this new dataset. Report the accuracy. Comment Any differences.</font>

<font color='green'>BONUS: Try to find a prompt that answer this question: What is the mean salary of titanic surviror based on adult dataset.</font>

In [ ]:
adult = pd.read_csv("hf://datasets/meghana/adult_income_dataset/adult.csv")
adult.info()

titanic = df